# Setup environment

In [ ]:
# python3 -m venv .venv or python -m venv .venv
# source .venv/bin/activate or .venv/Scripts/activate
# pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Recap Dictionary & Class

## Dictionary

In [ ]:
dictionary = {
    "x": "1",
    "y": "2",
}
dictionary

In [ ]:
dictionary["x"]

In [ ]:
dictionary["y"]

## Class

In [ ]:
class Test:
    def __init__(self, **kwargs):
        self.x = kwargs.get("x")
        self.y = kwargs.get("y")

In [ ]:
test_instance = Test(**dictionary)
test_instance

In [ ]:
test_instance.x

In [ ]:
test_instance.y

# Langchain Concept

## 1. LLM

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.base import Runnable

def chat_openai() -> Runnable:
    openai_api_key = os.getenv("OPENAI_API_KEY")
    llm = ChatOpenAI(
        openai_api_key=openai_api_key,
        model = "gpt-4o-mini",
        temperature=0
        )
    return llm

llm = chat_openai()
llm

In [ ]:
respone = llm.invoke("Hi!")
respone

In [ ]:
respone.content

## 2. Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# prompt template that have to put human name
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. My name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{query}")
])

prompt_template

In [ ]:
prompt_template.messages

## 3. Chain (& LCEL)

In [ ]:
chain = prompt_template | llm
chain

In [ ]:
response = chain.invoke({
    "name": "Pan",
    "query": "What is the capital of France?"
})
response

In [ ]:
response.content

## 4. Pydantic Parser

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from typing import List, Dict, Any
from pydantic import BaseModel, Field


class Output(BaseModel):
    answer: str = Field(description="The answer to the user's question.")
    reasoning: str = Field(description="The reasoning behind the answer.")

parser = PydanticOutputParser(pydantic_object=Output)
print(parser.get_format_instructions())

In [ ]:
from langchain_core.messages import HumanMessage

prompt_template.messages.append(HumanMessage(
    content=parser.get_format_instructions()
))
prompt_template.messages

### Option 1

In [ ]:
chain = prompt_template | llm | parser
chain

In [ ]:
response = chain.invoke({
    "name": "Pan",
    "query": "What is the capital of France?"
})
response

### Option 2

In [ ]:
chain = prompt_template | llm
chain

In [ ]:
response = chain.invoke({
    "name": "Pan",
    "query": "What is the capital of France?"
})
response

## 5. Memory

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = []
messages

In [ ]:
query = HumanMessage(
    content = "Hi, my name is Pan"
)
response = llm.invoke([query])
response

### Test Memory

In [ ]:
response_test = llm.invoke("Do you know my name?")
response_test

### Solution

In [ ]:
messages.append(response)
messages

In [ ]:
response = llm.invoke([*messages, HumanMessage(content="Do you know my name?")])
response

## 6. Sequential Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from operator import itemgetter

# Chain translation
template_translation = ChatPromptTemplate([
    ("system", "You are an expert translator from English to Thai."),
    ("human", "Translate the following sentence to Thai: {query}")
])

class TranslationOutput(BaseModel):
    translation: str = Field(description="The translated sentence.")

parser = PydanticOutputParser(pydantic_object=TranslationOutput)

template_translation.messages.append(HumanMessage(
    content=parser.get_format_instructions()
))

chain_translation = (template_translation | llm | parser).with_config({"run_name": "translation"})
chain_translation

In [ ]:
# Chain พ่อขุนรามคำแหง
template_tone_adjustment = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. You can adjust the tone of the text."),
    ("human", "Adjust the tone of the following text to be more 'พ่อขุนรามคำแหง' style: {query}"),
])

class ToneAdjustmentOutput(BaseModel):
    adjusted_text: str = Field(description="The text with the adjusted tone.")

parser = PydanticOutputParser(pydantic_object=ToneAdjustmentOutput)

template_tone_adjustment.messages.append(HumanMessage(
    content=parser.get_format_instructions()
))

chain_tone_adjustment = (template_tone_adjustment | llm | parser).with_config({"run_name": "tone_adjustment"})
chain_tone_adjustment

### Test Translation

In [ ]:
response = chain_translation.invoke("Hello, my name is Pan. I'm coming from Thailand.")
response

### Test Tone Adjustment

In [ ]:
response = chain_tone_adjustment.invoke({
    "query": response.translation
})
response

### Consolidate Chain

#### Option 1

In [ ]:
chain_final = chain_translation | chain_tone_adjustment
chain_final

In [ ]:
response = chain_final.invoke("Hello, my name is Pan. I'm coming from Thailand.")
response

#### Option 2

In [ ]:
chain_final = (
    {"query": itemgetter("query")}
    | chain_translation
    | (lambda output: {"query": output.translation})
    | chain_tone_adjustment
)

chain_final

In [ ]:
response = chain_final.invoke({
    "query": "Hello, my name is Pan. I'm coming from Thailand."
})
response

## 7. Debugging

In [ ]:
from langchain.globals import set_debug

set_debug(True)

response = chain_final.invoke({
    "query": "Hello, my name is Pan. I'm coming from Thailand."
})
set_debug(False)

response